In [46]:
import torch, os
import pandas as pd
import numpy as np
from numpy import random as npr
import torch.nn as nn

In [47]:
def which_mat(TTS, choice, n = [7, 7*2, 7*4, 7*4*2, 7*4*3], p = [0], channel_use = 1):
    """
    Funciton which can be easilly calle din order to implament which_mat as part of a larger function
    """
    return get_mat(TTS, n[choice[0]], p[choice[1]], channel_use)

def get_mat(TTS, n,p = 0, channel_use = 1):
    """
    Funciton that takes in 
    TTS Tesnor of Time Serieses, and given 
    n number of period to use and
    p  last index period to use, 
    return a covariance matrix of the data for that period in terms of the values in chanel channel_use
    """
    
    d1, d2 = TTS[0], TTS[1]
    #insert function to partition TTS correctly
    
    #creating cova matrix estimation desired
    d = [d1, d1]
    return pd.DataFrame({"S%d" %i: d for i,d in  enumerate(d)}).cov()


def cov_matrix_loss(A,B, type = 1):
    """
    Get distance between matracies based either on component wise distance or using eigenvalues
    """
    if type == 1:
        return torch.sqrt(torch.sum(torch.pow(torch.log(torch.eig(A-B))),2))
    else:
        ind = np.triu_indices(2)
        loss = nn.MSELoss()
        return loss(A[ind], B[ind]) 

In [48]:
class TS_based_simple(nn.modules):
    def __init__(self, kernel = 5, in_channels =4,  hidden_size = 128):
        """
        
        
        """
        super(TS_based_picker, self).__init__(self):
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels = channels_in, out_channels = hidden_size, 
                      stride = 2,kernel_size = kernel, padding = pads )
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d(2)
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size, out_channels = (hidden_size/2), 
                      kernel_size = (kenrnel-2), stride = 2, padding = (pads-1) )
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d(2)
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels = (hidden_size/2), out_channels = (hidden_size/(2**2)), 
                      kernel_size = (kenrnel-2), stride = 2, padding = (pads-1) )
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d(2)
            )
        
        self.downconvLast = nn.Sequential(
            nn.Conv1d(in_channels, out_channels = num_outs, kernel_size = (kenrnel-2), padding = (pads-1))
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d()
            )
        
        self.picker = nn.Linear()
        
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data and downconvthat
        self.cl = self.downconvLast(torch.cat((self.c23, self.c3), dim=1))
        
        #concat TS based data to data representative of other info
        self.last = self.picker

class TS_based_wfactors(nn.modules):
    def __init__(self, num_outs, kernel = 5, channels_in =4, hidden_size = 128):
        """
        
        
        """
        super(TS_based_picker, self).__init__(self):
        
        pads  == kernel//2
        
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels = channels_in, out_channels = hidden_size, 
                      stride = 2,kernel_size = kernel, padding = pads )
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d(2)
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size, out_channels = (hidden_size/2), 
                      kernel_size = (kenrnel-2), stride = 2, padding = (pads-1) )
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d(2)
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels = (hidden_size/2), out_channels = (hidden_size/(2**2)), 
                      kernel_size = (kenrnel-2), stride = 2, padding = (pads-1) )
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d(2)
            )
        
        self.downconvLast = nn.Sequential(
            nn.Conv1d(in_channels, out_channels = num_outs, kernel_size = (kenrnel-2), padding = (pads-1))
            nn.BatchNorm1d()
            nn.ReLU()
            nn.MaxPool1d()
            )
        
        self.picker = nn.Linear()
        
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data and downconvthat
        self.cl = self.downconvLast(torch.cat((self.c23, self.c3), dim=1))
        
        #concat TS based data to data representative of other info
        self.last = self.picker
        

SyntaxError: invalid syntax (<ipython-input-48-a0b0dc34d812>, line 2)

In [ ]:
def  train(args):
    #setting up the packages
    torch.set_num_threads(5)
    npr.seed(args.seed)
    dir_save_to = "results/" + args.dir_name
    
    #setting up the model
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(cnn.parameters(), lr=args.learn_rate)
    
    #which model to implament
    if cnn is None:
        if args.model == "Mod1":
              mod_use = TS_based_simple(args.k_size, args.num_filters, num_colours, arg.num_in_chans)
        elif args.model == "Mod1wFactors":
              mod_use = TS_based__wfactors(args.k_size, args.num_filters, num_colours, arg.num_in_chans)

In [ ]:
args = {seed: 12345
        GPU = "TRUE"
        dir_name = "trial1"
        learn_rate = 0.005
        model_use = "Mod1"
        num_in_chans = 4
        args.k_size = 5
}

In [8]:
b = pd.read_csv("100stocks.csv")